In [3]:
!export CUDA_VISIBLE_DEVICES=0

'export' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.current_device()

0

In [8]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import torch
torch.cuda.is_available()

# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'
# # os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

# torch.cuda.set_device(0)
# print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
# import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()
    print(torch.multiprocessing.get_start_method())
#     torch.multiprocessing.set_start_method('spawn',force=True)
    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            # from train_whisperclass import Solver
            from Whisper_Biclass_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))

    for idx in range(0,5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}.yaml'#latest7k_4lstm.pth
        # setattr(paras, 'load', f'./LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/whisper-v2-15k-a-d.pth')
        #要用vag01要改load_ckpt
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)


        solver.load_data()
#         solver.print_model()
        solver.set_model()
        solver.exec()
        del solver

In [2]:
main()

spawn

Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-17']
Mozillacv11Dataset CTT5-1-17 found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: data_process, split: ['CTT5-5-17', 'CTT5-2-17', 'CTT5-3-17', 'CTT5-4-17']
Mozillacv11Dataset CTT5-5-17 found wav data: 25
Mozillacv11Dataset CTT5-2-17 found wav data: 38
Mozillacv11Dataset CTT5-3-17 found wav data: 21
Mozillacv11Dataset CTT5-4-17 found wav data: 25
text len: 109
remove None, then wav data: 109, text len: 109
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

C:\Users\naomi\AppData\Roaming\Python\Python38\site-packages\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:92: RuntimeWarning: Mean of empty slice.
  normed_slice = (vector - vector[:length].mean()) / np.sqrt(vector[:length].var() + 1e-7)
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\naomi\AppData\Roaming\Python\Python38\site-packages\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice
  normed_slice = (vector - vector[:length].mean()) / np.sqrt(vector[:length].var() + 1e-7)
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encount

total_loss tensor(75.4340, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(74.6837, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(74.2329, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.9117, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.7987, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.6904, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.5984, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.5089, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.3530, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.3235, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.1891, device='cuda:0', grad_fn=<AddBackward0>) 

self.tr_set: 109
total_loss tensor(73.1374, device='cuda:0',

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-17']
Mozillacv11Dataset CTT5-3-17 found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: data_process, split: ['CTT5-1-17', 'CTT5-2-17', 'CTT5-5-17', 'CTT5-4-17']
Mozillacv11Dataset CTT5-1-17 found wav data: 21
Mozillacv11Dataset CTT5-2-17 found wav data: 38
Mozillacv11Dataset CTT5-5-17 found wav data: 25
Mozillacv11Dataset CTT5-4-17 found wav data: 25
text len: 109
remove None, then wav data: 109, text len: 109
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-17']
Mozillacv11Dataset CTT5-4-17 found wav data: 25
text len: 25
remove None, then wav data: 25, text len: 25
[VAGDataset] path: data_process, split: ['CTT5-1-17', 'CTT5-2-17', 'CTT5-3-17', 'CTT5-5-17']
Mozillacv11Dataset CTT5-1-17 found wav data: 21
Mozillacv11Dataset CTT5-2-17 found wav data: 38
Mozillacv11Dataset CTT5-3-17 found wav data: 21
Mozillacv11Dataset CTT5-5-17 found wav data: 25
text len: 105
remove None, then wav data: 105, text len: 105
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-17']
Mozillacv11Dataset CTT5-5-17 found wav data: 25
text len: 25
remove None, then wav data: 25, text len: 25
[VAGDataset] path: data_process, split: ['CTT5-1-17', 'CTT5-2-17', 'CTT5-3-17', 'CTT5-4-17']
Mozillacv11Dataset CTT5-1-17 found wav data: 21
Mozillacv11Dataset CTT5-2-17 found wav data: 38
Mozillacv11Dataset CTT5-3-17 found wav data: 21
Mozillacv11Dataset CTT5-4-17 found wav data: 25
text len: 105
remove None, then wav data: 105, text len: 105
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

In [ ]:
# torch.cuda.device_count()
import torch
torch.cuda.is_available()
# torch.cuda.device_count() 

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv

In [ ]:
# 'data_process/CTT5-2-2/2CTT.wav'
# v1: ['我看到有一對兄妹感覺是想要偷吃餅乾然後那個哥哥踩的椅子好像快跌倒了這樣子然後還有媽媽在洗碗可是水潮都水水龍頭沒有關所以水一直漏出來這樣子OK應該這樣子吧然後窗外好像有一些東西']
# v2: ['我看到有一對兄妹感覺是想要偷吃餅乾然後那個哥哥踩的椅子好像快跌倒了這樣子然後還有媽媽在洗碗可是水槽的水 水龍頭沒有關所以水一直漏出來這樣子窗外好像有些東西']
# v3: ['看到有一對兄妹感覺是想要偷吃餅乾哥哥踩的椅子好像快跌倒了然後還有媽媽在洗碗可是水槽的水溫度沒有關所以水一直漏出來窗外好像有些東西']

# 'data_process/CTT5-1-2/33CTT.wav'
# v1: ['在內容內容你看到什麼就說什麼你覺得是什麼就是什麼沒有標準的答案隨便講一個兩個三個四個三個三個不講不做事的']
# v2: ['在內容內容一個兩個三個四個三個三個不講不做事的']
# v3: ['在內容內蒙一個兩個三個四個三個三個不講不做事的']

# 'data_process/CTT5-3/57CTT.wav'
# v1: ['有一個小孩子在拿櫃台那個櫃上的餅乾但是站在的椅子上會非常危險椅子有傾斜有跌下來的可能那一個媽媽在洗碗槽前面洗碗那洗碗槽裡水已經流下來流了滿地他自己並不知覺他繼續做他的工作']

# 'data_process/CTT5-4/21CTT.wav'
# v1: ['男孩子要拿東西板凳沒凳好倒下來這樣很危險還有媽媽在洗碗盤那水滿出來了其他的都沒']

# 'data_process/CTT5-4/6CTT.wav'
# v1: ['哥哥拿餅乾坐在電子椅子上,椅子快要倒下來了然後媽媽在洗這個餐具的時候水都已經漏出來了大概就是這樣子吧']

# 'data_process/CTT5-4/7CTT.wav'
# v1: ['看到了一個媽媽在洗碗然後水潮的水就滿出來了然後兩個小朋友在拿餅乾有一個餅乾就是椅子快掉下來就是快跌倒的樣子然後就這樣']

In [1]:
# decode about 30 sec
import torch
import os
import pandas as pd

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2Model
import soundfile as sf

# load model and processor
device = torch.device('cuda:0')
model_checkpoint = "TencentGameMate/chinese-wav2vec2-large" # distil-whisper/distil-large-v2
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_checkpoint)
# outofCUDA
model = Wav2Vec2Model.from_pretrained(model_checkpoint).to(device)
# whisper = WhisperModel.from_pretrained(model_checkpoint).to(device)
# model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="zh", task="transcribe")
model.config.output_attentions = True
model.config.output_hidden_states = True

path = "data_process/CTTsegment_remove"
transcriptionlist = pd.DataFrame()
for f in os.listdir(path):
    filepath = path + '/' + f
    if os.path.isfile(filepath) and f.find("wav") != -1:
        print("filepath: ", filepath)
        wav, sr = sf.read(filepath)
        temp = "data_process/CTTsegment_remove/temp.wav"
        sf.write(temp, wav, 16000)
        wav, sr = sf.read(temp)
        print(wav, sr)
        # import librosa    
        # wav, sr = librosa.load(filepath, sr=16000) # Downsample 44.1kHz to 8kHz
        # print(sr)
        # outofCUDA
        # input_features = processor(wav, sampling_rate=sr, return_tensors="pt").input_features.to(device)
        
        input_values = processor(wav, sampling_rate=sr, return_tensors="pt", padding="longest").input_values.to(device)  # Batch size 1
        print(input_values.shape)
        # retrieve logits
        logits = model(input_values).last_hidden_state
        
        # # take argmax and decode
        # predicted_ids = torch.argmax(logits, dim=-1)
        # transcription = processor.batch_decode(predicted_ids)

        # print(logits)

c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


filepath:  data_process/CTTsegment_remove/100CTT.wav
[0. 0. 0. ... 0. 0. 0.] 16000
torch.Size([1, 816304])
filepath:  data_process/CTTsegment_remove/10CTT.wav
[0. 0. 0. ... 0. 0. 0.] 16000
torch.Size([1, 355968])
filepath:  data_process/CTTsegment_remove/11CTT.wav
[0. 0. 0. ... 0. 0. 0.] 16000
torch.Size([1, 246208])
filepath:  data_process/CTTsegment_remove/12CTT.wav
[0. 0. 0. ... 0. 0. 0.] 16000
torch.Size([1, 570816])
filepath:  data_process/CTTsegment_remove/13CTT.wav
[0. 0. 0. ... 0. 0. 0.] 16000
torch.Size([1, 1961152])


In [ ]:
# decode about 30 sec
import torch
import os
from pydub import AudioSegment
import pandas as pd

from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperModel
import soundfile as sf

import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# from datasets import Audio, load_dataset

# load model and processor
device = torch.device('cuda:0')
model_checkpoint = "openai/whisper-large-v2"  # distil-whisper/distil-large-v2
processor = WhisperProcessor.from_pretrained(model_checkpoint)
# outofCUDA
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
whisper = WhisperModel.from_pretrained(model_checkpoint).to(device)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="zh", task="transcribe")
model.config.output_attentions = True
model.config.output_hidden_states = True

# load streaming dataset and read first audio sample
# common_voice_test_transcription = load_dataset("mozilla-foundation/common_voice_13_0", "zh-TW", split="test")
# ds = load_dataset("mozilla-foundation/common_voice_13_0", "zh-TW", split="test", streaming=True)
# ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
# input_speech = next(iter(ds))["audio"]

path = "data_process/CTTsegment_diarization_overlap"
transcriptionlist = pd.DataFrame()
for f in os.listdir(path):
    filepath = path + '/' + f
    if os.path.isfile(filepath) and f.find("wav") != -1:
        print("filepath: ", filepath)
        wav, sr = sf.read(filepath)
        temp = "data_process/CTTsegment_diarization_overlap/temp.wav"
        sf.write(temp, wav, 16000)
        wav, sr = sf.read(temp)

        # import librosa    
        # wav, sr = librosa.load(filepath, sr=16000) # Downsample 44.1kHz to 8kHz
        # print(sr)
        # outofCUDA
        input_features = processor(wav, sampling_rate=sr, return_tensors="pt").input_features.to(device)
        
        # generate token ids # outofCUDA
        predicted_ids = model.generate(input_features).to(device)
        # print(type(predicted_ids))
        # decode token ids to text
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        print(transcription)
                
        # input_features = input_features.to(device)
        # predicted_ids = predicted_ids.to(device)

        # outputs = whisper(input_features, decoder_input_ids=predicted_ids, output_attentions=True, output_hidden_states=True)
        # print("hidden state size: ", len(outputs.last_hidden_state), outputs.last_hidden_state.shape)
        # # print("hidden state: ", outputs.last_hidden_state)
        # print("attention size: ", len(outputs.cross_attentions), outputs.cross_attentions[0].shape)
        # print("mel spectrogram: ", input_features.shape)
        # print("len: ", len(transcription[0]))

        # mel_spectrum = input_features.cpu().detach().numpy()
        # cross_attention_weights = outputs.cross_attentions[0].squeeze(0).cpu().detach().numpy()

        # # from sklearn.preprocessing import MinMaxScaler
        # # scaler = MinMaxScaler()
        # # cross_attention_weights = scaler.fit_transform(cross_attention_weights.mean(axis=0))
        
        # cross_attention_weights = cross_attention_weights.mean(axis=0)

        # # heat map / attention map
        # plt.figure(figsize=(10, 6))
        # sns.heatmap(cross_attention_weights, cmap='viridis', annot=True, fmt=".2f", xticklabels=False, yticklabels=False, linewidths = 0.05)
        # plt.xlabel('Key Sequence')
        # plt.ylabel('Query Sequence')
        # plt.title('Cross Attention Weights Visualization')
        # # plt.tight_layout()
        # plt.show()

        temp = pd.DataFrame({"audio": f, "transcription": transcription})
        # temp = pd.DataFrame({"audio": f, "predicted_ids": predicted_ids})
        transcriptionlist = pd.concat([transcriptionlist, temp])
        # f_withoutwav = f.replace(".wav", "")
        # torch.save(predicted_ids, f'data_process/CTTsegment_30/predicted_ids/{f_withoutwav}.pt')
        
transcriptionlist.to_csv("data_process/CTTsegment_diarization_overlap/transcription-v2.csv", index=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


filepath:  data_process/Lu_CTTdeletion_o_d/002_1.wav


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


['四層桃']
filepath:  data_process/Lu_CTTdeletion_o_d/002_2.wav
['媽媽在洗碗筷,可能是想一些心事吧']
filepath:  data_process/Lu_CTTdeletion_o_d/002_3.wav
['洗的有咖啡杯啊盘子啊这厨房倒是很干净他这样子这个水喷的到处都是他还穿着围裙拿个抹布在擦擦这个盘子好像在想心事的样子所以他的孩子在这里这样他都不知道水溢出来他也不知道琉璃台应该是讲小孩子吗小孩子够是不是要拿点心吃不是他弟弟就是哥哥']
filepath:  data_process/Lu_CTTdeletion_o_d/002_4.wav
['讲小孩子吗?小孩子要够是不是要拿点心吃不是他弟弟,是哥哥']
filepath:  data_process/Lu_CTTdeletion_o_d/002_5.wav
['夾糖 方糖的夾子算盤手風琴 廣球拍 羽毛球拍這個是圓規表 圓規尺 圓規...喔不是 這個量尺量角度的應該量度 電梯 手扶器這個就是那個要測量的那個腳架我們會叫它三腳架 倒把對 這個這也是樂器之一嘛對']
filepath:  data_process/Lu_CTTdeletion_o_d/002_6.wav
['我會叫它三角架 棗靶對 這個這個是樂器之一嘛對應該是不太認識它應該講是豎琴輪椅這個是窗台上的花架嗯 很好這個是河馬這不就是河馬嗎我們會叫海馬蘑菇這個是那個愛斯基摩人的住的屋子什麼什麼什麼我不會說這個圓龜']
filepath:  data_process/Lu_CTTdeletion_o_d/002_7.wav
['我還以為是吉姆人的住的屋子什麼的,我不會知道,原歸']
filepath:  data_process/Lu_CTTdeletion_o_d/003_1.wav
['猪没差人,野狗子,老虎,想不起来,动物,几三不三动物呢,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,猪,

In [ ]:
import torch
from transformers import AutoFeatureExtractor, WhisperForAudioClassification
import os
import pandas as pd
import soundfile as sf

import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from datasets import load_dataset

feature_extractor = AutoFeatureExtractor.from_pretrained("sanchit-gandhi/whisper-large-v3")
model = WhisperForAudioClassification.from_pretrained("sanchit-gandhi/whisper-large-v3").to('cuda')

path = "data_process/CTTsegment"
transcriptionlist = pd.DataFrame()
for f in os.listdir(path):
    filepath = path + '/' + f
    if os.path.isfile(filepath) and f.find("wav") != -1:
        print("filepath: ", filepath)
        wav, sr = sf.read(filepath)
        input_features = feature_extractor(wav, sampling_rate=sr, return_tensors="pt").input_features.to('cuda')

        # generate token ids
        outputs = model(input_features, output_attentions=True, output_hidden_states=True)
        print("hidden state size: ", len(outputs.hidden_states), outputs.hidden_states[0].shape)
        print("hidden state: ", outputs.hidden_states[-1])
        print("attention size: ", len(outputs.attentions), outputs.attentions[0].shape)
        print("predict_label: ", model.config.id2label[torch.argmax(outputs.logits).item()])
        print("loss: ", outputs.loss)
        print("mel spectrogram: ", input_features.shape)

        mel_spectrum = input_features.cpu().detach().numpy()
        attention_weight = outputs.attentions[0]

        mel_spect = librosa.feature.melspectrogram(y=wav, sr=16000, n_fft=400, hop_length=160, n_mels=128)
        print(mel_spect.shape)

        # mel = np.exp(mel_spectrum[0])
        mel_db = librosa.power_to_db(mel_spect, ref=np.max)
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mel_db, x_axis='s', y_axis='mel', sr=sr, hop_length=160, n_fft=400, cmap='viridis')
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        plt.show()

        
        # from torch import nn
        # conv1 = nn.ConvTranspose2d(in_channels=20, out_channels=20, kernel_size=2, stride=2, padding=0).cuda()
        # outputs = conv1(attention_weight).squeeze(0).cpu().detach().numpy()
        import torch.nn.functional as F
        upsampled_output = F.interpolate(attention_weight, size=(3000, 3000), mode='bilinear', align_corners=False).squeeze(0).cpu().detach().numpy()

        from validation_visualize import visualize_head, visualize_heads, visualize_head_average, visualize_heads_average, visualize_heads_resized
        # target_shape = mel_spectrum[0].shape
        # print("target_shape: ", target_shape)
        # visualize_heads_resized(attention_weight, cols=4, target_shape=target_shape)
        visualize_heads(upsampled_output, cols=4)

        # 將 self-attention map 的尺寸調整為與 mel spectrogram 相同的大小
        # target_shape = mel_spectrum[0].shape
        # attention_resized = np.resize(attention_weight.mean(axis=0), target_shape)

        # # 將 self-attention map 和 mel spectrogram 疊加在一起（這裡使用加權平均）
        # alpha = 0.5  # 加權平均的權重
        # overlay = alpha * attention_resized + (1 - alpha) * mel_spectrum[0]

        # 繪製疊加後的圖像
        plt.imshow(upsampled_output.mean(axis=0), cmap='viridis', aspect='auto', origin='lower')
        plt.title('Whisper Encoder Self-Attention Map Overlay')
        plt.colorbar()  
        plt.xlabel('Time')
        plt.ylabel('Mel Spectrogram Frequency')
        plt.show()
        
        # # Mel Spectrogram
        # plt.subplots(figsize=(10, 6))
        # librosa.display.specshow(mel_db, y_axis='mel', sr=16000, hop_length=160, n_fft=400, cmap='viridis')
        # plt.colorbar(format='%+2.0f dB', label='Intensity')

        # # Cross-Attention Weights
        # plt.imshow(attention_resized, cmap='viridis', aspect='auto', origin='lower', alpha=0.1)
        # plt.title('Whisper Encoder Self-Attention Map Overlay')
        # plt.xlabel('Time')
        # plt.ylabel('Mel Spectrogram Frequency')
        # plt.colorbar()  
        # plt.show()

In [1]:
import torch
torch.cuda.is_available()

# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'
# # os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

# torch.cuda.set_device(0)
# print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
# import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()
    print(torch.multiprocessing.get_start_method())
#     torch.multiprocessing.set_start_method('spawn',force=True)
    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            # from train_whisperclass import Solver
            from Whisper_Biclass_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))

    for idx in range(0,5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}.yaml'#latest7k_4lstm.pth
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'#latest7k_4lstm.pth
        
        setattr(paras, 'load', f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/whisper-v2-15k-r-a-final.pth') # whisper15k_with_id.pth
        #要用vag01要改load_ckpt
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)


        solver.load_data()
#         solver.print_model()
        solver.set_model()
        # solver.exec()
        # solver.validate(idx)
        solver.visualization()
        del solver

In [2]:
main()

spawn

Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
[INFO] Load ckpt from LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0/whisper-v2-15k-r-a-final.pth, restarting at step 15001 
fc.weight
fc.bias
output 0K] Valid step - 1/17tensor([0.6694], device='cuda:0')

d:\Lulu\Research\0709\LAS_Mandarin_PyTorch-master\Whisper_Biclass_train.py:612: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\captum\attr\_utils\visualization.py:338: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\captum\_utils\gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
d:\Lulu\Research\0709\LAS_Mandarin_PyTorch-master\Whisper_Biclass_train.py:671: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
d:\Lulu\Research\0709\LAS_Mandarin_PyTorch-master\Whisper_Biclass_train.py:707: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt

output 0K] Valid step - 2/17tensor([0.6563], device='cuda:0')


d:\Lulu\Research\0709\LAS_Mandarin_PyTorch-master\Whisper_Biclass_train.py:612: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\captum\attr\_utils\visualization.py:338: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
d:\Lulu\Research\0709\LAS_Mandarin_PyTorch-master\Whisper_Biclass_train.py:671: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


output tensor([0.9293], device='cuda:0')
output tensor([0.7947], device='cuda:0')
output tensor([0.2388], device='cuda:0')
output tensor([0.4073], device='cuda:0')
output tensor([0.6801], device='cuda:0')
output tensor([0.1474], device='cuda:0')
output tensor([0.7532], device='cuda:0')
